# 명령 작업으로 학습 스크립트 실행

Azure Machine Learning용 Python SDK를 사용하여 스크립트를 명령 작업으로 제출할 수 있습니다. 작업을 사용하면 기계 학습 모델을 학습할 때 입력 매개 변수 및 출력을 쉽게 추적할 수 있습니다.

## 시작하기 전에

이 Notebook에서 코드를 실행하려면  **최신 버전의 azureml-ai-ml** 패키지가 필요합니다. 아래 셀의 명령을 실행하여 이 패키지가 설치되어 있는지 확인합니다.

> **고**:
> **azure-ai-ml** 패키지가 설치되지 않은 경우 를 실행 `pip install azure-ai-ml` 하여 설치합니다.

In [ ]:
## 작업 영역에 연결

필요한 SDK 패키지를 설치했으므로 작업 영역에 연결할 수 있습니다.

작업 영역에 연결하려면 식별자 매개 변수(구독 ID, 리소스 그룹 이름 및 작업 영역 이름)가 필요합니다. 리소스 그룹 이름 및 작업 영역 이름이 이미 채워져 있습니다. 명령을 완료하려면 구독 ID만 필요합니다.

필요한 매개 변수를 찾으려면 Studio의 오른쪽 위에 있는 구독 및 작업 영역 이름을 클릭합니다. 오른쪽에 창이 열립니다.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 구독 ID를 복사하고 **YOUR-SUBSCRIPTION-ID** 를 복사한 값으로 바꿉니다. </p>

## 명령 작업 시작

아래 셀을 실행하여 당뇨병을 예측하는 분류 모델을 학습시킵니다. 모델은 **src** 폴더에서 찾을 수 있는 **train-model-parameters.py** 스크립트를 실행하여 학습됩니다. **diabetes.csv** 파일을 학습 데이터로 사용합니다. 

- `code`: 실행할 스크립트를 포함하는 폴더를 지정합니다.
- `command`: 정확히 실행할 항목을 지정합니다.
- `environment`: 명령을 실행하기 전에 컴퓨팅에 설치하는 데 필요한 패키지를 지정합니다.
- `compute`: 명령을 실행하는 데 사용할 컴퓨팅을 지정합니다.
- `display_name`: 개별 작업의 이름입니다.
- `experiment_name`: 작업이 속한 실험의 이름입니다.

터미널에서 스크립트를 테스트하는 데 사용되는 명령은 아래 작업 구성의 명령과 동일합니다. 

In [ ]:
# enter details of your AML workspace
subscription_id = "YOUR-SUBSCRIPTION-ID"
resource_group = "rg-dp100-labs"
workspace = "mlw-dp100-labs"

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

## Initiate a command job

Run the cell below to train a classification model to predict diabetes. The model is trained by running the **train-model-parameters.py** script that can be found in the **src** folder. It uses the **diabetes.csv** file as the training data. 

- `code`: specifies the folder that includes the script to run.
- `command`: specifies what to run exactly.
- `environment`: specifies the necessary packages to be installed on the compute before running the command.
- `compute`: specifies the compute to use to run the command.
- `display_name`: the name of the individual job.
- `experiment_name`: the name of the experiment the job belongs to.

Note that the command used to test the script in the terminal is the same as the command in the configuration of the job below. 

In [ ]:
from azure.ai.ml import command

# configure job

job = command(
    code="./src",
    command="python train-model-parameters.py --training_data diabetes.csv",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-script",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)